In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
import pickle
import joblib
import os

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Functions

In [15]:
def save_model(model, model_path):
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)

def load_model(model_path):
    with open(model_path, 'rb') as f:
        return pickle.load(f)

def load_dataset(folder_name):
    with open(f'../data/{folder_name}/train.pkl', 'rb') as f:
        X_train, y_train = pickle.load(f)
    with open(f'../data/{folder_name}/val.pkl', 'rb') as f:
        X_val, y_val = pickle.load(f)
    with open(f'../data/{folder_name}/test.pkl', 'rb') as f:
        X_test, y_test = pickle.load(f)
    
    scaler_y = None
    if folder_name in ['norm', 'hot_norm']:
        with open(f'../data/scaler_y.pkl', 'rb') as f:
            scaler_y = pickle.load(f)
    
    return X_train, X_val, X_test, y_train, y_val, y_test, scaler_y

def evaluate_model(model, model_path, X_train, X_val, X_test, y_train, y_val, y_test, scaler_y=None, dataset_name=''):
    performances = []
    data_splits = [(X_train, y_train, 'train'), (X_val, y_val, 'val'), (X_test, y_test, 'test')]
    
    for x, y, nsplit in data_splits:
        preds = model.predict(x)
        r2 = r2_score(y, preds)
        rmse = np.sqrt(mean_squared_error(y, preds))
        mae = mean_absolute_error(y, preds)
        
        model_name = model.__class__.__name__
        if model_name == 'KNeighborsRegressor':
            model_name += f" (k={model.n_neighbors}, weights={model.weights}, metric={model.metric})"
        if model_name == 'DecisionTreeRegressor':
            model_name += f" (max_depth={model.max_depth}, min_samples_split={model.min_samples_split}, min_samples_leaf={model.min_samples_leaf})"
        if model_name == 'RandomForestRegressor':
            model_name += f" (n_estimators={model.n_estimators}, max_depth={model.max_depth}, min_samples_split={model.min_samples_split}, min_samples_leaf={model.min_samples_leaf}, max_features={model.max_features}, bootstrap={model.bootstrap})"

        performances.append({
            'model': model_name,
            'dataset': dataset_name,
            'split': nsplit,
            'rmse': rmse.round(4),
            'mae': mae.round(4),
            'r2': r2.round(4),
            'inversed': False,
            'model_path': model_path
        })
        
        if scaler_y is not None:
            preds_original = scaler_y.inverse_transform(preds.reshape(-1, 1)).flatten()
            y_original = scaler_y.inverse_transform(y.reshape(-1, 1)).flatten()
            
            r2_original = r2_score(y_original, preds_original)
            rmse_original = np.sqrt(mean_squared_error(y_original, preds_original))
            mae_original = mean_absolute_error(y_original, preds_original)
            
            performances.append({
                'model': model_name,
                'dataset': dataset_name,
                'split': nsplit,
                'rmse': rmse_original.round(4),
                'mae': mae_original.round(4),
                'r2': r2_original.round(4),
                'inversed': True,
                'model_path': model_path
            })
    
    return performances

# Custom grid search function
def custom_grid_search_knn(X_train, y_train, param_grid, dataset_name):
    model_dir = '../models'
    os.makedirs(model_dir, exist_ok=True)
    
    for params in param_grid:
        knn = KNeighborsRegressor(**params)
        knn.fit(X_train, y_train)
        
        model_path = os.path.join(model_dir, f'knn_{params["n_neighbors"]}_neighbors_{params["weights"]}_{params["metric"]}_{dataset_name}.pkl')
        save_model(knn, model_path)
        print(f"Trained and saved KNeighborsRegressor model with parameters {params} to {model_path}")

# Custom grid search function for Decision Tree
def custom_grid_search_dt(X_train, y_train, param_grid, dataset_name):
    model_dir = '../models'
    os.makedirs(model_dir, exist_ok=True)
    
    for params in param_grid:
        dt = DecisionTreeRegressor(**params)
        dt.fit(X_train, y_train)
        
        model_path = os.path.join(model_dir, f'dt_{params["max_depth"]}_depth_{params["min_samples_split"]}_minsplit_{params["min_samples_leaf"]}_minleaf_{dataset_name}.pkl')
        save_model(dt, model_path)
        print(f"Trained and saved DecisionTreeRegressor model with parameters {params} to {model_path}")

# Custom grid search function for Random Forest
def custom_grid_search_rf(X_train, y_train, param_grid, dataset_name):
    model_dir = '../models'
    os.makedirs(model_dir, exist_ok=True)
    
    for params in param_grid:
        model_path = os.path.join(model_dir, f'rf_{params["n_estimators"]}_estimators_{params["max_depth"]}_depth_{params["min_samples_split"]}_minsplit_{params["min_samples_leaf"]}_minleaf_{params["max_features"]}_features_{params["bootstrap"]}_bootstrap_{dataset_name}.pkl')
        
        if os.path.exists(model_path):
            print(f"Model with parameters {params} already exists. Loading model from {model_path}")
            rf = load_model(model_path)
        else:
            rf = RandomForestRegressor(**params)
            rf.fit(X_train, y_train)
            save_model(rf, model_path)
            print(f"Trained and saved RandomForestRegressor model with parameters {params} to {model_path}")
        
        performances = evaluate_model(rf, model_path, X_train, X_val, X_test, y_train, y_val, y_test, scaler_y=scaler_y, dataset_name=dataset_name)
        all_performances.extend(performances)


# Fit models

# Median model

In [4]:
# Load the dataset
X_train, X_val, X_test, y_train, y_val, y_test, scaler_y = load_dataset('hot')

# Calculate the median value of the training set responses
median_value = y_train.median()

# Define the model name for easy reference
model_name = 'median'

performances = []

# Loop through each dataset (training, validation, test)
for y, nsplit in zip([y_train, y_val, y_test], ['train', 'val', 'test']):
    # Create an array filled with the mean value for predictions
    median_predictions = [median_value] * y.shape[0]
    
    # Calculate the RMSE using the true values and the constant mean predictions
    rmse = np.sqrt(mean_squared_error(y, median_predictions))
    
    # Calculate the R2 score using the true values and the constant mean predictions
    r2 = r2_score(y, median_predictions)
    
    # Calculate the MAE using the true values and the constant mean predictions
    mae = mean_absolute_error(y, median_predictions)
    
    # Append the performance metrics to the list
    performances.append({
        'model': model_name,
        'split': nsplit,
        'rmse': rmse.round(4),
        'mae': mae.round(4),
        'r2': r2.round(4)
    })

performances_df = pd.DataFrame(performances)
performances_df

,model,split,rmse,mae,r2
0,median,train,192.9508,119.3824,-0.0896
1,median,val,192.9799,119.8874,-0.0857
2,median,test,194.6283,120.1703,-0.0885


# KNN

In [9]:
# Define parameter grid for KNN
param_grid_knn = [{'n_neighbors': k, 'weights': w, 'metric': m} for k in [5, 15, 20, 25, 30, 50] for w in ['uniform', 'distance'] for m in ['euclidean', 'manhattan']]

# Initialize a list to store all performances
all_performances = []

# List of dataset names and their corresponding folder names
dataset_names = ['hot_norm']

# Load each dataset and perform custom grid search
for dataset_name in dataset_names:
    X_train, X_val, X_test, y_train, y_val, y_test, scaler_y = load_dataset(dataset_name)
    custom_grid_search_knn(X_train, y_train, param_grid_knn, dataset_name)
    
    # Evaluate all saved models
    for params in param_grid_knn:
        model_path = os.path.join('../models', f'knn_{params["n_neighbors"]}_neighbors_{params["weights"]}_{params["metric"]}_{dataset_name}.pkl')
        knn = load_model(model_path)
        performances = evaluate_model(knn, model_path, X_train, X_val, X_test, y_train, y_val, y_test, scaler_y=scaler_y, dataset_name=dataset_name)
        all_performances.extend(performances)

# Display performances
for performance in all_performances:
    print(performance)

# Save performances to a CSV file
perf_df = pd.DataFrame(all_performances)
perf_df.to_csv('../data/knn_performances.csv', index=False)


Trained and saved KNeighborsRegressor model with parameters {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'euclidean'} to ../models/knn_5_neighbors_uniform_euclidean_hot_norm.pkl
Trained and saved KNeighborsRegressor model with parameters {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'manhattan'} to ../models/knn_5_neighbors_uniform_manhattan_hot_norm.pkl
Trained and saved KNeighborsRegressor model with parameters {'n_neighbors': 5, 'weights': 'distance', 'metric': 'euclidean'} to ../models/knn_5_neighbors_distance_euclidean_hot_norm.pkl
Trained and saved KNeighborsRegressor model with parameters {'n_neighbors': 5, 'weights': 'distance', 'metric': 'manhattan'} to ../models/knn_5_neighbors_distance_manhattan_hot_norm.pkl
Trained and saved KNeighborsRegressor model with parameters {'n_neighbors': 15, 'weights': 'uniform', 'metric': 'euclidean'} to ../models/knn_15_neighbors_uniform_euclidean_hot_norm.pkl
Trained and saved KNeighborsRegressor model with parameters {'n_neighbors'

# Descision tree

In [10]:

# Define parameter grid for Decision Tree
param_grid_dt = [{'max_depth': d, 'min_samples_split': s, 'min_samples_leaf': l} for d in [None, 10, 20, 30, 40] for s in [2, 10, 20, 30] for l in [1, 5, 10, 20]]

# Initialize a list to store all performances
all_performances = []

# List of dataset names and their corresponding folder names
dataset_names = ['hot']

# Load each dataset and perform custom grid search
for dataset_name in dataset_names:
    X_train, X_val, X_test, y_train, y_val, y_test, scaler_y = load_dataset(dataset_name)
    
    # Decision Tree Grid Search
    custom_grid_search_dt(X_train, y_train, param_grid_dt, dataset_name)
    
    # Evaluate all saved Decision Tree models
    for params in param_grid_dt:
        model_path = os.path.join('../models', f'dt_{params["max_depth"]}_depth_{params["min_samples_split"]}_minsplit_{params["min_samples_leaf"]}_minleaf_{dataset_name}.pkl')
        dt = load_model(model_path)
        performances = evaluate_model(dt, model_path, X_train, X_val, X_test, y_train, y_val, y_test, scaler_y=scaler_y, dataset_name=dataset_name)
        all_performances.extend(performances)

# Display performances
for performance in all_performances:
    print(performance)

# Turn into pd.DataFrame
perf_df = pd.DataFrame(all_performances)

# Save performances to a CSV file
perf_df.to_csv('../data/dt_performances.csv', index=False)


Trained and saved DecisionTreeRegressor model with parameters {'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1} to ../models/dt_None_depth_2_minsplit_1_minleaf_hot.pkl
Trained and saved DecisionTreeRegressor model with parameters {'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 5} to ../models/dt_None_depth_2_minsplit_5_minleaf_hot.pkl
Trained and saved DecisionTreeRegressor model with parameters {'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 10} to ../models/dt_None_depth_2_minsplit_10_minleaf_hot.pkl
Trained and saved DecisionTreeRegressor model with parameters {'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 20} to ../models/dt_None_depth_2_minsplit_20_minleaf_hot.pkl
Trained and saved DecisionTreeRegressor model with parameters {'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 1} to ../models/dt_None_depth_10_minsplit_1_minleaf_hot.pkl
Trained and saved DecisionTreeRegressor model with parameters {'max_d

# Random forest

## First random search to give a hint to the parameters for the extensive grid search

In [11]:
# Make a grid search for Random Forest
# Define the model
rf = RandomForestRegressor()

# Define the parameter grid
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Load the dataset
X_train, X_val, X_test, y_train, y_val, y_test, scaler_y = load_dataset(dataset_name)

# Grid search
grid_search = RandomizedSearchCV(rf, param_grid, n_iter=10, cv=5, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vec

[CV] END bootstrap=False, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END bootstrap=False, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END bootstrap=False, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END bootstrap=False, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END bootstrap=False, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=10; total time=   0.5s


/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vec

[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=10; total time=   0.7s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=10; total time=   0.7s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=10; total time=   0.7s
[CV] END bootstrap=False, max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.5s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=10; total time=   0.7s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=10; total time=   0.7s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=10; total time=   0.7s
[CV] END bootstrap=True, max_depth=30, max_features=

/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_

Best parameters found:  {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': False}


Best parameters found:  {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': False}

## Exhaustive grid search

In [16]:
# Define parameter grid for Random Forest
param_grid_rf = [
    {'n_estimators': n, 'max_depth': d, 'min_samples_split': s, 'min_samples_leaf': l, 'max_features': f, 'bootstrap': b} 
    for n in [25, 50, 75]  # including the best n_estimators
    for d in [None, 20, 30, 40]  # starting with best max_depth and adding more options
    for s in [5, 10, 15]  # including the best min_samples_split
    for l in [1, 2, 5]  # including the best min_samples_leaf
    for f in ['sqrt', 'log2']  # starting with best max_features and adding more options
    for b in [False, True]  # including the best bootstrap
]

# Initialize a list to store all performances
all_performances = []

# List of dataset names and their corresponding folder names
dataset_names = ['hot']

# Load each dataset and perform custom grid search
for dataset_name in dataset_names:
    X_train, X_val, X_test, y_train, y_val, y_test, scaler_y = load_dataset(dataset_name)
    
    # Random Forest Grid Search
    custom_grid_search_rf(X_train, y_train, param_grid_rf, dataset_name)

# Turn into pd.DataFrame
perf_df = pd.DataFrame(all_performances)

# Save performances to a CSV file
perf_df.to_csv('../data/rf_performances.csv', index=False)

/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Trained and saved RandomForestRegressor model with parameters {'n_estimators': 25, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False} to ../models/rf_25_estimators_None_depth_5_minsplit_1_minleaf_sqrt_features_False_bootstrap_hot.pkl


/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Trained and saved RandomForestRegressor model with parameters {'n_estimators': 25, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': True} to ../models/rf_25_estimators_None_depth_5_minsplit_1_minleaf_sqrt_features_True_bootstrap_hot.pkl


/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


...

Trained and saved RandomForestRegressor model with parameters {'n_estimators': 75, 'max_depth': 40, 'min_samples_split': 15, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'bootstrap': True} to ../models/rf_75_estimators_40_depth_15_minsplit_5_minleaf_sqrt_features_True_bootstrap_hot.pkl


/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Trained and saved RandomForestRegressor model with parameters {'n_estimators': 75, 'max_depth': 40, 'min_samples_split': 15, 'min_samples_leaf': 5, 'max_features': 'log2', 'bootstrap': False} to ../models/rf_75_estimators_40_depth_15_minsplit_5_minleaf_log2_features_False_bootstrap_hot.pkl


/work/Exam/DataScienceExam/env/airbnb/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Trained and saved RandomForestRegressor model with parameters {'n_estimators': 75, 'max_depth': 40, 'min_samples_split': 15, 'min_samples_leaf': 5, 'max_features': 'log2', 'bootstrap': True} to ../models/rf_75_estimators_40_depth_15_minsplit_5_minleaf_log2_features_True_bootstrap_hot.pkl
